```
This file is part of sscx-connectome-manipulations.

SPDX-License-Identifier: Apache-2.0
Copyright (c) 2024 Blue Brain Project/EPFL
```

# Structural comparison of simplified SSCx connectomes
This is an auxiliary notebook for analysing the simplified 1st..5th-order connectomes, which

 - sets up structural comparator
 - writes .json config file for running structural comparisons using `sbatch`

ℹ️ Part of reproduction of simplified connectomes experiment described in [Pokorny et al. (2024)](https://doi.org/10.1101/2024.05.24.593860)

<u>Requirements</u>:
- [Connectome-Manipulator](https://github.com/BlueBrain/connectome-manipulator) (Python venv)
- [SSCx network model](https://doi.org/10.5281/zenodo.8026353) (=original circuit)
- Rewired (simplified) circuits, see `SSCx_rewiring.ipynb` (also available in Zenodo dataset)

In [1]:
# Initialization

""" Global imports """
import json
import os

""" Local imports """
from connectome_manipulator.connectome_comparison import structural_comparator

In [2]:
def default_structcomp_config(output_path, base_name, circuit_specs):
    """Generates a default structural comparator config dict w/o any specific comparison."""
    assert isinstance(circuit_specs, list) and len(circuit_specs) == 2, 'ERROR: Two circuits required for comparison!'

    structcomp_config = {}
    structcomp_config['working_dir'] = os.path.join(output_path, base_name)
    structcomp_config['out_dir'] = structcomp_config['working_dir']
    structcomp_config['circuits'] = {str(i): {'circuit_config': circuit_specs[i]['config'],
                                              'circuit_name': circuit_specs[i]['name']} for i in range(2)}
    structcomp_config['plot_types'] = []

    return structcomp_config

def upper_first(name):
    return name[0].upper() + name[1:]

def syn_class_mapping(sel):
    assert sel in [None, 'EE', 'EI', 'IE', 'II'], 'ERROR: Synapse class error!'
    cl_map = {'E': 'EXC', 'I': 'INH'}
    if sel is None:
        return {}, {}
    else:
        return {'synapse_class': cl_map[sel[0]]}, {'synapse_class': cl_map[sel[1]]}

def add_ConnProb_to_config(structcomp_config, group_by, hex_column='hex0', syn_class_sel='EE', skip_empty_groups=False):
    """Adds "connection probability" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    assert group_by in ['layer', 'mtype'], f'ERROR: Grouping by "{group_by}" not supported!'
    pre_cl_sel, post_cl_sel = syn_class_mapping(syn_class_sel)
    comp_dict = {'name': f'ConnPer{upper_first(group_by)}_{upper_first(hex_column)}{"" if syn_class_sel is None else syn_class_sel}',
                 'fct': {'source': 'connectivity',
                         'kwargs': {'group_by': group_by,
                                    'skip_empty_groups': skip_empty_groups,
                                    'sel_src': {'node_set': hex_column, **pre_cl_sel},
                                    'sel_dest': {'node_set': hex_column, **post_cl_sel}}},
                         'res_sel': ['nsyn_conn', 'conn_prob'],
                         'range_prctile': 100,
                         'fig_size': (11, 3),
                         'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def add_MeanProps_to_config(structcomp_config, props_list, group_by, hex_column='hex0', syn_class_sel='EE', skip_empty_groups=False):
    """Adds "connection properties (mean) per layer" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    assert isinstance(props_list, list) and len(props_list) > 0, "ERROR: Non-empty list of properties required!"
    assert group_by in ['layer', 'mtype'], f'ERROR: Grouping by "{group_by}" not supported!'
    pre_cl_sel, post_cl_sel = syn_class_mapping(syn_class_sel)
    comp_dict = {'name': f'PropsPer{upper_first(group_by)}_{upper_first(hex_column)}{"" if syn_class_sel is None else syn_class_sel}',
                         'fct': {'source': 'properties',
                                 'kwargs': {'group_by': group_by,
                                            'skip_empty_groups': skip_empty_groups,
                                            'sel_src': {'node_set': hex_column, **pre_cl_sel},
                                            'sel_dest': {'node_set': hex_column, **post_cl_sel},
                                            'fct': 'np.mean'}},
                                 'res_sel': props_list,
                                 'range_prctile': 100,
                                 'fig_size': (11, 3),
                                 'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def add_StdProps_to_config(structcomp_config, props_list, group_by, hex_column='hex0', syn_class_sel='EE', skip_empty_groups=False):
    """Adds "connection properties (std) per layer" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    assert isinstance(props_list, list) and len(props_list) > 0, "ERROR: Non-empty list of properties required!"
    assert group_by in ['layer', 'mtype'], f'ERROR: Grouping by "{group_by}" not supported!'
    pre_cl_sel, post_cl_sel = syn_class_mapping(syn_class_sel)
    comp_dict = {'name': f'PropsStdPer{upper_first(group_by)}_{upper_first(hex_column)}{"" if syn_class_sel is None else syn_class_sel}',
                         'fct': {'source': 'properties',
                                 'kwargs': {'group_by': group_by,
                                            'skip_empty_groups': skip_empty_groups,
                                            'sel_src': {'node_set': hex_column, **pre_cl_sel},
                                            'sel_dest': {'node_set': hex_column, **post_cl_sel},
                                            'fct': 'np.std'}},
                                 'res_sel': props_list,
                                 'range_prctile': 100,
                                 'fig_size': (11, 3),
                                 'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def add_AdjMat_to_config(structcomp_config, hex_column='hex0', syn_class_sel='EE'):
    """Adds "adjacency matrix" comparison to structural comparator config."""
    assert 'plot_types' in structcomp_config, 'ERROR: Plot types missing!'
    pre_cl_sel, post_cl_sel = syn_class_mapping(syn_class_sel)
    comp_dict = {'name': f'Adjacency_{upper_first(hex_column)}{"" if syn_class_sel is None else syn_class_sel}',
                         'fct': {'source': 'adjacency',
                                 'kwargs': {'sel_src': {'node_set': hex_column, **pre_cl_sel},
                                            'sel_dest': {'node_set': hex_column, **post_cl_sel}}},
                                 'res_sel': ['adj', 'adj_cnt'],
                                 'range_prctile': 95,
                                 'fig_size': (11, 3),
                                 'fig_file': {'format': 'png', 'dpi': 600}}
    structcomp_config['plot_types'].append(comp_dict)

def export_structcomp_config(structcomp_config, config_path, config_name=''):
    """Writes structural comparator config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    base_name = os.path.split(structcomp_config['working_dir'])[-1]
    fn = f'structcomp_config{config_name}__{base_name}__{structcomp_config["circuits"]["0"]["circuit_name"]}_vs_{structcomp_config["circuits"]["1"]["circuit_name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(structcomp_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

def print_launch_cmd(config_path, config_fn, force_recomp=False):
    if force_recomp:
        run_cmd = f"sbatch run_struct_comparison.sh {config_fn} --force-recomp-circ1 --force-recomp-circ2"
    else:
        run_cmd = f"sbatch run_struct_comparison.sh {config_fn}"
    print(f"# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:")
    print(f"cd {config_path}")
    print(run_cmd)

def load_config_from_file(path, fn):
    config_file = os.path.join(path, fn)
    with open(config_file, 'r') as f:
        structcomp_config = json.load(f)
    return structcomp_config

### Selection of circuits to compare

ℹ️ Set paths below pointing to the original as well as rewired (simplified) circuits

In [3]:
# Circuit base name
circuit_name_base = 'SSCx-HexO1-Release'

# Original circuit
circuit_name_orig = 'Orig'
circuit_config_orig = '/gpfs/bbp.cscs.ch/project/proj83/jira-tickets/NSETM-1948-extract-hex-O1/data/O1_data/circuit_config.json' # SONATA config (.json)
circuit_spec_orig = {'name': circuit_name_orig, 'config': circuit_config_orig}

# Simplified circuits
order_list = [1, 2, 3, 4, 5]
circuit_names_manip = [f'Order-{_ord}' for _ord in order_list]
circuit_configs_manip = [f'/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/Zenodo/SSCx-connectome-manipulation-data/simplified_connectomes/circuits/{circuit_name_base}__ConnRewireOrder{_ord}Hex0EE/circuit_config.json' for _ord in order_list]
circuit_specs_manip = [{'name': nm, 'config': cfg} for nm, cfg in zip(circuit_names_manip, circuit_configs_manip)]

### Configuration of structural comparison

ℹ️ Set config and output paths below

In [5]:
# Output paths
output_path = '/gpfs/bbp.cscs.ch/data/scratch/proj83/home/pokorny/Zenodo/SSCx-connectome-manipulation-data/simplified_connectomes/structural_comparator/'
config_path = '../configs'  # Must exist!

# Selection of column & synapse class
hex_column = 'hex0'  # hex0...central column
config_name = ''  # Config name specified (default: '')

# Selection of physiological properties to compare
props_list = ['conductance', 'decay_time', 'delay', 'depression_time', 'facilitation_time', 'n_rrp_vesicles', 'syn_type_id', 'u_syn']

In [7]:
# List of structural comparisons of original vs. simplified connectomes
# Notes: syn_class_sel ... Must be one of [None, 'EE', 'EI', 'IE', 'II']

for idx in range(len(order_list)):
    structcomp_config = default_structcomp_config(output_path, circuit_name_base, [circuit_spec_orig, circuit_specs_manip[idx]])
    add_ConnProb_to_config(structcomp_config, group_by='layer', hex_column=hex_column, syn_class_sel='EE', skip_empty_groups=False)
    add_ConnProb_to_config(structcomp_config, group_by='mtype', hex_column=hex_column, syn_class_sel='EE', skip_empty_groups=True)
    add_MeanProps_to_config(structcomp_config, props_list=props_list, group_by='mtype', hex_column=hex_column, syn_class_sel='EE', skip_empty_groups=True)
    add_StdProps_to_config(structcomp_config, props_list=props_list, group_by='mtype', hex_column=hex_column, syn_class_sel='EE', skip_empty_groups=True)
    add_AdjMat_to_config(structcomp_config, hex_column=hex_column, syn_class_sel=None)
    export_structcomp_config(structcomp_config, config_path, config_name)

Config file structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-1.json written to ../configs
Config file structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-2.json written to ../configs
Config file structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-3.json written to ../configs
Config file structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-4.json written to ../configs
Config file structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-5.json written to ../configs


### Run structural comparison

ℹ️ Configure the SLURM script `run_struct_comparison.sh` according to the used computation system and run below LAUNCH COMMANDS for extracting/computing the comparison results

ℹ️ All structural comparison results are also contained in the Zenodo dataset

In [9]:
for _ord in order_list:
    print_launch_cmd(config_path, f'structcomp_config{config_name}__SSCx-HexO1-Release__Orig_vs_Order-{_ord}.json', force_recomp=False)
    print()
    if _ord == order_list[0]:
        print('WAIT UNTIL "ORIG" RESULTS ARE READY BEFORE LAUNCHING THE OTHERS!')
        print()

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd ../configs
sbatch run_struct_comparison.sh structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-1.json

WAIT UNTIL "ORIG" RESULTS ARE READY BEFORE LAUNCHING THE OTHERS!

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd ../configs
sbatch run_struct_comparison.sh structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-2.json

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd ../configs
sbatch run_struct_comparison.sh structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-3.json

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd ../configs
sbatch run_struct_comparison.sh structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-4.json

# LAUNCH COMMAND [CHECK ALLOCATION TIME!!]:
cd ../configs
sbatch run_struct_comparison.sh structcomp_config__SSCx-HexO1-Release__Orig_vs_Order-5.json



---
After successful comparison runs: The comparison results/figures are available in the specified output folders

---